In [ ]:
# Cell 1: Import required libraries and functions
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import pandas as pd
from datetime import datetime
import os

# Cell 2: Visualization functions
def plot_results(results_df):
    """Create visualizations and summary of results"""
    # Display recommendations summary
    print("\nRecommendation Summary:")
    rec_counts = results_df['Recommendation'].value_counts()
    display(rec_counts)

    # Plot recommendation distribution
    plt.figure(figsize=(10, 6))
    rec_order = [
        'STRONG_BUY', 'BUY', 'WEAK_BUY', 
        'HOLD', 
        'WEAK_SELL', 'SELL', 'STRONG_SELL'
    ]
    # Only include categories that exist in the data
    existing_categories = [cat for cat in rec_order if cat in rec_counts.index]
    
    if existing_categories:
        sns.countplot(x='Recommendation', data=results_df, order=existing_categories)
        plt.title('Stock Recommendations Distribution')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("No recommendation data to plot")

    # Ensure we have required columns for displays
    required_cols = ['Symbol', 'Last_Price', 'Recommendation']
    missing_cols = [col for col in required_cols if col not in results_df.columns]
    
    if missing_cols:
        print(f"Missing required columns for display: {missing_cols}")
        return results_df, results_df  # Return empty DataFrames as placeholders
        
    # Optional columns to display if available
    display_cols = ['Symbol', 'Last_Price', 'Recommendation']
    for col in ['RSI', 'ADX', 'Sentiment', 'ML_Probability', 'ML_Accuracy']:
        if col in results_df.columns:
            display_cols.append(col)
    
    # Show top buy recommendations
    buy_stocks = results_df[results_df['Recommendation'].isin(['STRONG_BUY', 'BUY'])].sort_values('Last_Price', ascending=True)
    if 'ML_Probability' in results_df.columns:
        buy_stocks = results_df[results_df['Recommendation'].isin(['STRONG_BUY', 'BUY'])].sort_values('ML_Probability', ascending=False)
    
    print("\nTOP BUY RECOMMENDATIONS:")
    if len(buy_stocks) > 0:
        display(buy_stocks[display_cols])
    else:
        print("No buy recommendations found")

    # Show top sell recommendations
    sell_stocks = results_df[results_df['Recommendation'].isin(['STRONG_SELL', 'SELL'])].sort_values('Last_Price', ascending=True)
    if 'ML_Probability' in results_df.columns:
        sell_stocks = results_df[results_df['Recommendation'].isin(['STRONG_SELL', 'SELL'])].sort_values('ML_Probability', ascending=True)
    
    print("\nTOP SELL RECOMMENDATIONS:")
    if len(sell_stocks) > 0:
        display(sell_stocks[display_cols])
    else:
        print("No sell recommendations found")

    # Only create scatter plot if we have the necessary columns
    if 'Sentiment' in results_df.columns and 'ML_Probability' in results_df.columns:
        try:
            # Create scatter plot of Sentiment vs ML_Probability colored by Recommendation
            plt.figure(figsize=(12, 8))
            recommendation_colors = {
                'STRONG_BUY': 'darkgreen',
                'BUY': 'green',
                'WEAK_BUY': 'lightgreen',
                'HOLD': 'blue',
                'WEAK_SELL': 'salmon',
                'SELL': 'red',
                'STRONG_SELL': 'darkred'
            }

            for rec in recommendation_colors:
                subset = results_df[results_df['Recommendation'] == rec]
                if len(subset) > 0:
                    plt.scatter(subset['Sentiment'], subset['ML_Probability'], 
                              c=recommendation_colors[rec], label=rec, s=100, alpha=0.7)

            for i, row in results_df.iterrows():
                plt.annotate(row['Symbol'], 
                            (row['Sentiment'], row['ML_Probability']),
                            xytext=(5, 5), textcoords='offset points')

            plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.3)
            plt.axvline(x=0, color='gray', linestyle='--', alpha=0.3)
            plt.xlabel('Sentiment Score')
            plt.ylabel('ML Probability (Higher = More Bullish)')
            plt.title('Stock Analysis: Sentiment vs ML Prediction')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            plt.savefig(os.path.join(output_dir, f"sentiment_ml_plot_{timestamp}.png"))
            plt.show()
        except Exception as e:
            print(f"Error creating scatter plot: {e}")
    else:
        print("Cannot create scatter plot: missing required columns (Sentiment and/or ML_Probability)")
    
    return buy_stocks, sell_stocks

def save_and_display_results(results_df):
    """Save results to files and create styled display"""
    # ... existing code ...

# Cell 3: Main analysis functions
def analyze_stock(symbol, show_details=True):
    """Analyze a single stock with detailed output"""
    # ... existing code ...

def analyze_multiple_stocks(symbols, max_stocks=None):
    """Analyze multiple stocks and return a DataFrame of results"""
    # Cell 12: Save results and create a styled dataframe
def save_and_display_results(results_df):
    """Save results to files and create styled display with better error handling"""
    if len(results_df) == 0:
        print("No results to save or display")
        return results_df.style  # Return empty styled DataFrame
        
    # Create output directory if it doesn't exist
    output_dir = os.getenv('OUTPUT_DIR', './results')
    os.makedirs(output_dir, exist_ok=True)

    # Save results to CSV
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_file = os.path.join(output_dir, f"sp500_analysis_{timestamp}.csv")
    results_df.to_csv(output_file, index=False)
    print(f"\nResults saved to: {output_file}")
    
    # Create HTML report (with error handling for missing columns)
    try:
        # Count recommendations
        rec_counts = {
            'STRONG_BUY': len(results_df[results_df['Recommendation'] == 'STRONG_BUY']),
            'BUY': len(results_df[results_df['Recommendation'] == 'BUY']), 
            'HOLD': len(results_df[results_df['Recommendation'] == 'HOLD']),
            'SELL': len(results_df[results_df['Recommendation'] == 'SELL']),
            'STRONG_SELL': len(results_df[results_df['Recommendation'] == 'STRONG_SELL'])
        }
        
        # Get buy and sell recommendations
        buy_recs = results_df[results_df['Recommendation'].isin(['STRONG_BUY', 'BUY'])]
        sell_recs = results_df[results_df['Recommendation'].isin(['STRONG_SELL', 'SELL'])]
        
        html_report = f"""
        <html>
        <head>
            <title>S&P 500 Analysis Report - {datetime.now().strftime('%Y-%m-%d')}</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 40px; }}
                table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
                th, td {{ border: 1px solid #ddd; padding: 8px; text-align: center; }}
                th {{ background-color: #4CAF50; color: white; }}
                .buy {{ color: green; font-weight: bold; }}
                .sell {{ color: red; font-weight: bold; }}
                .hold {{ color: blue; font-weight: bold; }}
                .summary {{ margin: 20px 0; }}
            </style>
        </head>
        <body>
            <h1>S&P 500 Analysis Report</h1>
            <p>Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            
            <div class="summary">
                <h2>Summary</h2>
                <p>Total Stocks Analyzed: {len(results_df)}</p>
                <ul>
                    <li>Strong Buy: {rec_counts['STRONG_BUY']}</li>
                    <li>Buy: {rec_counts['BUY']}</li>
                    <li>Hold: {rec_counts['HOLD']}</li>
                    <li>Sell: {rec_counts['SELL']}</li>
                    <li>Strong Sell: {rec_counts['STRONG_SELL']}</li>
                </ul>
            </div>
            
            <h2>Top Buy Recommendations</h2>
            {buy_recs.to_html(classes='data', index=False) if len(buy_recs) > 0 else "<p>No buy recommendations found.</p>"}
            
            <h2>Top Sell Recommendations</h2>
            {sell_recs.to_html(classes='data', index=False) if len(sell_recs) > 0 else "<p>No sell recommendations found.</p>"}
        </body>
        </html>
        """
        
        html_file = os.path.join(output_dir, f"analysis_report_{timestamp}.html")
        with open(html_file, 'w') as f:
            f.write(html_report)
        
        print(f"HTML report saved to: {html_file}")
    except Exception as e:
        print(f"Error creating HTML report: {e}")
    
    # Style DataFrame for display
    def color_recommendation(val):
        if val == 'STRONG_BUY':
            return 'background-color: darkgreen; color: white'
        elif val == 'BUY':
            return 'background-color: green; color: white'
        elif val == 'WEAK_BUY':
            return 'background-color: lightgreen'
        elif val == 'HOLD':
            return 'background-color: lightblue'
        elif val == 'WEAK_SELL':
            return 'background-color: salmon'
        elif val == 'SELL':
            return 'background-color: red; color: white'
        elif val == 'STRONG_SELL':
            return 'background-color: darkred; color: white'
        return ''

    # Return styled DataFrame
    try:
        return results_df.style.applymap(color_recommendation, subset=['Recommendation'])
    except Exception as e:
        print(f"Error styling DataFrame: {e}")
        return results_df  # Return unstyles DataFrame as fallback